# Test ADC wheel

This notebook demonstrates how to use the various atmospheric dispersion correctors in METIS. Note that the action of an adc is currently restricted to a transmission loss. The implementation of the geometric differential refraction residuals will follow at a later stage.

In [1]:
import numpy as np

import scopesim as sim
sim.bug_report()

# Edit this path if you have a custom install directory, otherwise comment it out.
sim.link_irdb("../../../../")

Python:
3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]

Installed Python packages:
           anisocado: 0.3.0
         astar-utils: 0.3.3
             astropy: 6.1.7
      beautifulsoup4: 4.13.3
            docutils: 0.19
               httpx: 0.28.1
                lxml: 5.3.2
          matplotlib: 3.10.3
      more-itertools: 10.3.0
               numpy: 1.26.4
           packaging: 24.1
               pooch: 1.8.2
              pyyaml: 6.0.1
               scipy: 1.14.1
            scopesim: 0.10.0b4
       scopesim_data: could not be loaded.
  scopesim_templates: 0.6.2a0
         skycalc-ipy: 0.5.3
             synphot: 1.5.0
                tqdm: 4.66.5

Installed IRDB packages:

Operating System info:
   System: Windows
  Release: 11
  Version: 10.0.26100
  Machine: AMD64


If you haven't got the instrument packages yet, uncomment the following cell

In [2]:
# sim.download_packages(["METIS", "ELT", "Armazones"]) 

In [3]:
cmd = sim.UserCommands(use_instrument="METIS", set_modes=["img_lm"])

The ADC to use is defined by `"!OBS.adc"`. This can be set to `false` when no ADC is in the path. The default for LM band imaging is

In [4]:
cmd["!OBS.adc"]

'const_90'

In [5]:
src = sim.source.source_templates.empty_sky()

In [6]:
metis = sim.OpticalTrain(cmd)

The effect `metis["adc_wheel"]` works the same way as e.g. `metis["filter_wheel"]`. The following ADCs are now available (yes, there's only one) and can be selected with `metis["adc_wheel"].change_adc()` as demonstrated below.

In [7]:
metis["adc_wheel"].get_table()

name,max_transmission
str8,float64
const_90,0.9


Run a simulation with the ADC in the path:

In [8]:
metis.observe(src, update=True)
implane_adc = metis.image_planes[0].data

Now remove the ADC from the path by changing to `False`. Run the simulation without the ADC:

In [9]:
metis["adc_wheel"].change_adc(False)

In [10]:
metis.observe(src, update=True)
implane_no_adc = metis.image_planes[0].data

Compare the image plane simulated with and without the ADC. The ratio should be equal to the throughput of the ADC (90 per cent):

In [11]:
med_adc = np.median(implane_adc)
med_no_adc = np.median(implane_no_adc)
print("With ADC:    {:.1f}".format(np.median(med_adc)))
print("Without ADC: {:.1f}".format(np.median(med_no_adc)))
print("Ratio:       {:.1f}".format(med_adc/med_no_adc))

With ADC:    242637.3
Without ADC: 269597.0
Ratio:       0.9
